# Hypothesis Testing Using Northwind Database




In this research we will use Northwind Database to test hypothesis through experimental design.


## The Scientific Method And Experimental Design

<img src='The+Scientific+Method.jpg' height=70% width=70%>

Here's is the general structure of experimental design:

**1. Make an observation**

* Observe a phenomenon and formulate a question we want to test.

**2. Examine the research**

* Find out if there is an existing research that answers our question.
* *For the purposes of this project, we are going to skip this step.*

**3. Form a hypothesis**

* Formulate an Alternative Hypothesis (our educated guess to the test question) and a Null Hypothesis (the opposite to the Alternative Hypothesis).

**4. Conduct an experiment**

* Account for mistakes and randomness and run a well-structured experiment

**5. Analyze experimental results**

* Define if the outcomes of our experiment are statistically significant.

**6. Draw conclusions**

* Based on the findings and analysis of our experiment, either reject the Null Hypothesis or fail to reject the Null Hypothesis.


## Northwind Database

For this research, we will be using the Northwind Traders database. The database features a fictionary company, and was created in 2000 by Microsoft to demonstrate their SQL Server technology. 

<img src='Northwind_ERD.png' height=100% width=100%>

## Make An Observation

To formulate our hypothesis, we first need to take a closer look into the data contained in the Northwind database and make general observations.

In [ ]:
#import the necessary libraries
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import inspect

#connect to the database and check whether the table names correspond to the picture above
engine = create_engine("sqlite:///Northwind_small.sqlite", echo=True)
Session = sessionmaker(bind=engine)
session = Session()
inspector = inspect(engine)
print(inspector.get_table_names())

Now let's take a closer look at each table to gain a better understanding of the Northwind Traders company.

In [ ]:
#find out how many employees work for the company
df = pd.read_sql_query('''SELECT * FROM Employee''', engine)
df

In [ ]:
#find out how many customers the company works with
df = pd.read_sql_query('''SELECT * FROM Customer''', engine)
df.tail()

In [ ]:
#check the number of orders the company has processed
df = pd.read_sql_query('''SELECT * FROM [Order] ORDER BY ID DESC LIMIT 5''', engine)
df

In [ ]:
#understand the type of goods that the company works with
df = pd.read_sql_query('''SELECT * FROM Category''', engine)
df

In [ ]:
#see what products the company operates with 
df = pd.read_sql_query('''SELECT * FROM Product''', engine)
df

In [ ]:
#identify the suppliers the company works with
df = pd.read_sql_query('''SELECT * FROM Supplier''', engine)
df

In [ ]:
#check how many shippers the company has
df = pd.read_sql_query('''SELECT * FROM Shipper''', engine)
df

In [ ]:
#determine if the company offers any discounts
df = pd.read_sql_query('''SELECT DISTINCT Discount FROM OrderDetail''', engine)
df

**Findings:**
* Northwind Traders is a company that employs 9 people in the US and UK.
* They offer 77 food products in 8 categories.
* The company works with 29 suppliers worldwide and ships to 88 customers worldwide using the services of 3 shipper companies.
* Nothwind Traders processed 11,077 orders.
* They offer 10 discount categories.

## Forming A Hypothesis

Based on observations of the Northwind Traders database, we will perform 4 tests to validate the following hypotheses:

**Test 1**

Does discount amount have a statistically significant effect on the quantity of a product in an order? If so, at what level(s) of discount?

> $H_{0}$: *Discount amount does not have a statistically significant effect on the quantity of a product in an order.* 

> $H_{1}$: *Discount amount has a statistically significant effect on the quantity of a product in an order.*

**Test 2**

Is the amount of money spent by customers gender-related? Do women spend more money than men?

> $H_{0}$: *Male customers spend more money than female customers.* 

> $H_{1}$: *Female customers spend more money than male customers.*

**Test 3**

Is it more profitable for the company to sell to domestic or international customers?

> $H_{0}$: *Domestic customers are more profitable for the company.* 

> $H_{1}$: *International customers are more profitable for the company.*

**Test 4**

Is it more profitable for the company to work with domestic or international suppliers?

> $H_{0}$: *Domestic suppliers are more profitable for the company.* 

> $H_{1}$: *International suppliers are more profitable for the company.*

## Conducting Experiments And Analyzing Experimental Results

### Test 1

Our first question is whether the discount amount has a statistically significant effect on the quantity of a product in an order. If so, at what level(s) of discount?

> $H_{0}$: *Discount amount does not have a statistically significant effect on the quantity of a product in an order.* 

> $H_{1}$: *Discount amount has a statistically significant effect on the quantity of a product in an order.*

In [ ]:
#review the table that contains order discount
df1 = pd.read_sql_query('''SELECT * FROM OrderDetail''', engine)
print('\nTable info')
df1.info()
print('\nSample data')
df1.head()

In [ ]:
#find out how many products were sold at what discount
df1.Discount.value_counts()

Since 1%, 2%, 3%, 4% and 6% discount only occurred in 8 cases, we'll drop those rows and continue working with products sold at no discount or at a discount of 5%, 10%, 15%, 20%, 25%.

In [ ]:
#drop the lines with rarely occurring discount
df1_test = df1.drop(df1[(df1['Discount'] == 0.01) | (df1['Discount'] == 0.02) | (df1['Discount'] == 0.03) | (df1['Discount'] == 0.04) | (df1['Discount'] == 0.06)].index)
df1_test.Discount.value_counts()

In [ ]:
df1_test.Discount.hist(figsize=(5,3));

In [ ]:
df1_test.hist('Quantity', by = 'Discount', figsize=(7,7));

As we have 6 groups, we'll proceed with selecting ANOVA test, because it explains the total variance as combination of variances from different groups. F-distribution will become our test statistic. We set our significance level at 0.05, so we reject the null hypothesis if F>Fa.

In [ ]:
#import the libraries and define ANOVA model
import statsmodels.api as sm
from statsmodels.formula.api import ols
formula = 'Quantity ~ Discount'
lm = ols(formula, df1_test).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

Since our F value is higher than our alpha value, we can reject the null hypothesis. 

In [ ]:
#convert discount to dummy variables
discount_dummies = pd.get_dummies(df1_test['Discount'], prefix='discount')
df1_test = df1_test.drop(['Discount'], axis = 1)
df1_test = pd.concat([df1_test, discount_dummies], axis=1)
df1_test.head()

In [ ]:
#convert new columns to categoric values
df1_test['discount_0.0'] = df1_test['discount_0.0'].astype('category')
df1_test['discount_0.05'] = df1_test['discount_0.05'].astype('category')
df1_test['discount_0.1'] = df1_test['discount_0.1'].astype('category')
df1_test['discount_0.15'] = df1_test['discount_0.15'].astype('category')
df1_test['discount_0.2'] = df1_test['discount_0.2'].astype('category')
df1_test['discount_0.25'] = df1_test['discount_0.25'].astype('category')

Let's run ANOVA once again, now comparing various discount levels.

In [ ]:
formula = 'Quantity ~ C(discount_0.05) + C(discount_0.1) + C(discount_0.15) + C(discount_0.2) + C(discount_0.25)'
lm = ols(formula, df1_test).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

## Drawing Conclusions



Hypothesis
Obtaining the data
Test (Welch's T-test, one/two sample T-test, ANOVA)
Test statistic (p value)
Effect size
Conclusions